In [1]:
import pandas as pd
import numpy as np
from snowflake.sqlalchemy import URL
from datetime import datetime
import warnings
from dateutil.relativedelta import relativedelta

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

/Users/keyurpethad/anaconda3/lib/python3.11/site-packages/snowflake/connector/options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (11.0.0), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
from sqlalchemy import create_engine


class extractorFromQuery:
    def __init__(self):
        self.credentials = {
            "account": "qfi-niro",
            "user": "keyur",
            "password": "Keyurpethad@1706",
            "warehouse": "NIRO_ANALYTICS_WAREHOUSE",
        }

    def queryExecutor(self, query: str, *args):
        engine = create_engine(URL(**self.credentials))
        res = pd.read_sql_query(query, engine, params=args)
        return res

In [3]:
observation_date = datetime(2024, 1, 31)
observation_month = "jan'24"

In [4]:
master_calling = pd.read_excel("PL Loan Book Feb'24.xlsx", header=2)

In [5]:
master_calling.head()

,niro_user_id,Niro Opportunity ID,Disbursal Partner Name,Platform Partner Name,Lender Loan Id,Disbursement Month,Month Number,Disbursement Year,Phone Numbers,Disbursement Date,Presentation Lag,Gross PF Charged,PF Refund,Final Proc Fees,GST,Pre-EMI,Disbursed Amount,Loan Amount (all inclusive),Loan Amount Rank,Interest Rate,Tenor In Months,Tenor Rank,PF %,Cost of Funds,Cost of Funds (New),Spread,Monthly Emi,Monthly Income (Net Take Home),Income Rank,Income source,Self Employed,Nature of Business,# Years in Employment,Residence Ownership Type,Owned,Purpose of Loan,CIBIL Score,CIBIL Band,CIBIL Score Rank,Risk Band,PL & EL-PL,Disbursement TAT,City,State,Pin Code,Tier,Gender,Male,Female,Age,customer_router,CPV Initiated,Lead Name,Loan Status,Foreclosure Date,Foreclosure Details,NPA Details,NPA setteled Status,Write-off Month,Feb Presentation,Mar Presentation,Apr Presentation,May Presentation,Jun Presentation,Jul Presentation,Aug Presentation,Sep Presentation,Oct Presentation,Nov Presentation,Dec Presentation,Jan'23 Presentation,Feb'23 Presentation,Mar'23 Presentation,Apr'23 Presentation,May'23 Presentation,Jun'23 Presentation,July'23 Presentation,Aug'23 Presentation,Sep'23 Presentation,Oct'23 Presentation,Nov'23 Presentation,Dec'23 Presentation,Jan'24 Presentation,Feb'24 Presentation,Feb DPD,Mar DPD,Apr DPD,May DPD,Jun DPD,Jul DPD,Aug DPD,Sep DPD,Oct DPD,Nov DPD,Dec DPD,Jan'23 DPD,Feb'23 DPD,Mar'23 DPD,Apr'23 DPD,May'23 DPD,Jun'23 DPD,July'23 DPD,Aug'23 DPD,Sep'23 DPD,Oct'23 DPD,Nov'23 DPD,Dec'23 DPD,Jan'24 DPD,Mar DPD Rank,PayU'July'23 DPD,PayU'Aug'23 DPD,PayU'Sep'23 DPD,PayU'Oct'23 DPD,PayU'Nov'23 DPD,PayU'Dec'23 DPD,PayU'Jan'24 DPD,Sep DPD Rank,Jan - POS,Feb - POS,Mar - POS,Apr - POS,May - POS,Jun - POS,Jul - POS,Aug - POS,Sep - POS,Oct - POS,Nov - POS,Dec - POS,Jan'23 - POS,Feb'23 - POS,Mar'23 - POS,Apr'23 - POS,May'23 - POS,Jun'23 - POS,July'23 - POS,Aug'23 - POS,Sep'23 - POS,Oct'23 - POS,Nov'23 - POS,Dec'23 - POS,Jan'24 - POS,Int * Loan Amount (all Incl),CoF * Loan Amount (all Incl),Tenor * Loan Amount (all Incl),PF * Loan Amount (all Incl),Spread * Loan Amount (all Incl),FOIR,Ok/Problem Areas,Loan Count per customer,Repeat/Unique,Previous loan date of repeat,Previous loan amount of repeat,DPD 90+ Months,DPD 60+ Months,90+ Amount,Feb - Repaid,`,Apr - Repaid,May - Repaid,Jun - Repaid,Jul - Repaid,Aug - Repaid,Sep - Repaid,Oct - Repaid,Nov - Repaid,Dec - Repaid,Jan'23 - Repaid,Feb'23 - Repaid,Mar'23 - Repaid,Apr'23 - Repaid,May'23 - Repaid,Jun'23 - Repaid,July'23 - Repaid,Aug'23 - Repaid,Sep'23 - Repaid,Unnamed: 176,Unnamed: 177,Unnamed: 178,Unnamed: 179,Unnamed: 180
0,0a2c0c4e-e0bf-4e50-909e-25ad6272119e,0f100756-11ae-4472-bd29-4fdbd6c37fdb,liquiloans,quikr,499217,2021-12-31,1,2021,9900368067,2021-12-31,Yes,2366,0,2366,426.0,0.0,50005,52797,2,27.0,12,2,0.044813,17.4,17.4,9.6,5069.0,26000.0,3,Salaried,0,'-',1,Rented,0,Others,764,760+,8,CAT-D,EL-PL,< 24 hrs,Bangalore,KARNATAKA,560085,Tier 1,Female,0,1,39,PL_OLD_PREAPPROVED,Yes,VINUTHA HONNE GOWDA,Closed,NaT,NaT,NaN,No,-,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-,-,-,-,-,NaN,-,-,-,-,-,-,-,NaN,52797.0,48915.93250,44947.540981,40889.860653,36740.882518,32498.552375,28160.769803,23722.160542,19190.208334,14553.208334,9811.208334,4962.960521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1425519.0,918667.8,633564,2366.0,506851.2,0.44,OK Area,1,Unique,NaN,NaN,-,-,0.0,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-31,1.0
1,e85b616e-e54b-4d67-b929-5f4b612404e2,188ec580-5b20-435b-8b02-b1fd0c439c6b,liquiloans,quikr,500342,2022-01-31,2,2022,8765513171,2022-01-04,No,2366,0,2366,426.0,0.0,50005,52797,2,27.0,12,2,0.044813,17.4,17.4,9.6,5069.0,20000.0,2,Salaried,0,'-',1.5,Owned,1,Marriage,746,740-760,7,CAT-D,EL-PL,< 24 hrs,Lucknow,UTTA

In [6]:
master_calling.columns = master_calling.columns.str.lower().str.replace(' ', '_')

In [7]:
master_calling['pf'] = pd.to_numeric(master_calling["pf_%"].astype(str).str.replace(" ","").str.replace("%","").str.strip())

In [8]:
master_calling = master_calling[~master_calling.loan_status.isin(['Foreclosed', 'Closed', 'Settled'])]
master_calling = master_calling[master_calling.iloc[:,master_calling.columns.get_loc("feb_presentation"):master_calling.columns.get_loc(observation_month + "_presentation")].notna().sum(axis = 1) > 0].reset_index(drop=True)

In [9]:
master_calling.drop(columns = ['disbursement_date', 'interest_rate', 'income_source', 'residence_ownership_type', 'purpose_of_loan'], inplace=True)

In [10]:
master_calling.shape

(30663, 177)

#### Pre disbursal data

In [11]:
disbursed_cases = """
    select 
    la.id,
    ou.category,
    la.principal_amount,
    la.interest_rate,
    o.attributed_partner,
    o.propensity,
    o.offer_type,
    o.max_amount,
    tu.income_source,
    tu.purpose_of_loan,
    tu.residence_ownership_type,
    tu.marital_status,
    cpv.cpv_type,
    l.name as "lender_name",
    CASE
        when la.loan_disbursed_date_by_ops is null then date(la.disbursement_date)
        else date(la.loan_disbursed_date_by_ops)
    end as disbursement_date
    from niro_rds.niro_data.core__public__loan_application la
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__OFFERS o on o.id = la.offer_id
    left join NIRO_RDS.NIRO_DATA.core__public__offer_user_details ou on ou.offer_id = o.id
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__VERIFICATION_DETAILS vd on vd.id = la.verification_id 
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__TELE_PD_USER tu on vd.telepd_id::bigint = tu.id
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__CP_VERIFICATION cpv on cpv.id = vd.cpv_id::bigint
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__LENDERS l on l.id = o.lender_uid
    where la.loan_application_status in ('CLOSED','LOAN_EXECUTED') and la.is_deleted = False and o.is_deleted = False;
"""
total_disbursed_cases = extractorFromQuery().queryExecutor(disbursed_cases)
total_disbursed_cases.shape

(48038, 15)

In [12]:
total_disbursed_cases['residence_ownership_type'] = total_disbursed_cases.residence_ownership_type.str.lower().replace({'family_owned':'family owned'}).replace({'owned':'self owned'}).replace({'pg':'rented'}).replace({'room sharing':'rented'}).replace({'ancestral property':'family owned'})

In [13]:
disbursed_cases_with_cvattrs = """
            select * from (
            select 
            distinct la.id,
            pii.gender as acq_cibil_gender,
            pii.pin as acq_cibil_pincode,
            pii.calculated_age as acq_cibil_calculated_age,
            pii.state as acq_cibil_state,
            date_trunc("MONTH",date(od.created_at)) as createdate,
            od.current_salary as acq_current_salary,
            od.available_income as acq_available_income,
            od.foir_dlq as acq_foir_dlq, 
            od.foir_cibil as acq_foir_cibil, 
            od.foir_trd as acq_foir_trd, 
            od.foir_mean as acq_foir_mean, 
            od.new_foir as acq_new_foir, 
            od.naps_score as acq_naps_score,
            rank()over(partition by od.user_id order by od.created_at asc) as rnk,
            CASE
                when la.loan_disbursed_date_by_ops is null then date(la.disbursement_date)
                else date(la.loan_disbursed_date_by_ops)
            END as disbursement_date
            from NIRO_RDS.NIRO_DATA.CORE__PUBLIC__LOAN_APPLICATION la
            left join NIRO_RDS.NIRO_DATA.core__public__offer_user_details ou on ou.offer_id = la.offer_id
            left join ANALYTICS.OFFERS.offer_details od on od.user_id = la.user_id and od.category = ou.category
            left join ANALYTICS.OFFERS.personal_info pii on od.pii_id = pii.pii_id
            where la.loan_application_status in ('CLOSED','LOAN_EXECUTED') 
            and la.is_deleted = False
            and(la.is_deleted_at_source = false or la.is_deleted_at_source is null)
            and od.dec_reason = 'NOT DECLINED')
where rnk = 1;
"""
disbursed_cases = (
    extractorFromQuery()
    .queryExecutor(disbursed_cases_with_cvattrs)
    .drop(columns="disbursement_date")
)
disbursed_cases.shape

(42741, 15)

In [14]:
disbursed_cases = disbursed_cases.drop_duplicates(subset=["id"])
predisbursal_data = total_disbursed_cases.merge(disbursed_cases, on="id", how="left").rename(
    columns={"id": "niro_opportunity_id"}
)
predisbursal_data.drop_duplicates(subset=["niro_opportunity_id"], inplace=True)
predisbursal_data.shape

(48038, 29)

In [15]:
predisbursal_data['propensity'] = np.where(predisbursal_data['propensity'].isin(['HiiProp', 'High']), 'High',
                                  np.where(predisbursal_data['propensity'].isin(['MedProp', 'Med', 'Medium']), 'Medium',
                                  np.where(predisbursal_data['propensity'].isin(['LowProp', 'Low']), 'Low',None)))

In [16]:
predisbursal_data.head()

,niro_opportunity_id,category,principal_amount,interest_rate,attributed_partner,propensity,offer_type,max_amount,income_source,purpose_of_loan,residence_ownership_type,marital_status,cpv_type,lender_name,disbursement_date,acq_cibil_gender,acq_cibil_pincode,acq_cibil_calculated_age,acq_cibil_state,createdate,acq_current_salary,acq_available_income,acq_foir_dlq,acq_foir_cibil,acq_foir_trd,acq_foir_mean,acq_new_foir,acq_naps_score,rnk
0,9cde01df-344c-493c-bd03-ede315cf9df0,CAT-A,497008.0,18.9,betterhalf,Medium,PRE_APPROVED,480000.0,Salaried,Purchase,rented,SINGLE,REMOTE,payu,2023-08-10,Male,500075,38.0,ANDHRA PRADESH,2023-06-01,480000.0,91230.1,0.5,0.4,0.36,0.42,57.41,802.0,1.0
1,59af71ae-393f-4aa6-badb-79592ca9e5e4,CAT-D,85377.0,26.9,snapdeal,High,PRE_APPROVED,80000.0,Self Employed,Business Development,rented,MARRIED,PHYSICAL,liquiloans,2023-04-27,Male,211008,33.0,UTTAR PRADESH,2023-02-01,48000.0,27168.2,0.5,0.4,0.50,0.47,47.58,695.0,1.0
2,39132f29-5241-4bb1-bea6-025c86fbd938,CAT-C,177823.0,22.3,niro,High,PRE_APPROVED,170000.0,Salaried,Down Payment,family owned,MARRIED,PHYSICAL,liquiloans,2022-11-11,Male,500062,44.0,ANDHRA PRADESH,2022-08-01,48000.0,27162.5,0.4,0.5,0.50,0.47,56.59,608.0,1.0
3,d4b72fdb-b591-4dc0-b740-81f6ab29645c,CAT-D,53319.0,26.9,snapdeal,High,PRE_APPROVED,50000.0,Self Employed,Business Development,family owned,MARRIED,PHYSICAL,liquiloans,2023-04-29,Male,581320,37.0,KARNATAKA,2023-02-01,48000.0,23000.0,0.4,0.4,0.50,0.43,51.75,707.0,1.0
4,326e9fc3-5a65-418e-b67f-21bdabcdeada,CAT-D,53068.0,25.9,quikr,High,PRE_APPROVED,50000.0,Salaried,Purchase,family owned,MARRIED,REMOTE,muthoot,2023-03-20,Male,122001,24.0,HARYANA,2023-02-01,48000.0,23212.9,0.5,0.4,0.45,0.45,51.22,766.0,1.0


In [17]:
master_data = pd.merge(master_calling, predisbursal_data , on = 'niro_opportunity_id', how = 'inner')

In [18]:
master_data['disb_dt'] = pd.to_datetime(master_data['disbursement_date'], format= '%d-%b-%y', errors = 'coerce')
master_data['disb_yymm'] = master_data['disb_dt'].dt.to_period('M').dt.to_timestamp()

master_data['months_since_disb'] = (12*(observation_date.year - master_data['disb_yymm'].dt.year)+\
                           (observation_date.month - master_data['disb_yymm'].dt.month)+1).astype(int)

In [19]:
master_data.shape

(30663, 208)

In [20]:
master_data.tail()

,niro_user_id,niro_opportunity_id,disbursal_partner_name,platform_partner_name,lender_loan_id,disbursement_month,month_number,disbursement_year,phone_numbers,presentation_lag,gross_pf_charged,pf_refund,final_proc_fees,gst,pre-emi,disbursed_amount,loan_amount_(all_inclusive),loan_amount_rank,tenor_in_months,tenor_rank,pf_%,cost_of_funds,cost_of_funds_(new),spread,monthly_emi,monthly_income_(net_take_home),income_rank,self_employed,nature_of_business,#_years_in_employment,owned,cibil_score,cibil_band,cibil_score_rank,risk_band,pl_&_el-pl,disbursement_tat,city,state,pin_code,tier,gender,male,female,age,customer_router,cpv_initiated,lead_name,loan_status,foreclosure_date,foreclosure_details,npa_details,npa_setteled_status,write-off_month,feb_presentation,mar_presentation,apr_presentation,may_presentation,jun_presentation,jul_presentation,aug_presentation,sep_presentation,oct_presentation,nov_presentation,dec_presentation,jan'23_presentation,feb'23_presentation,mar'23_presentation,apr'23_presentation,may'23_presentation,jun'23_presentation,july'23_presentation,aug'23_presentation,sep'23_presentation,oct'23_presentation,nov'23_presentation,dec'23_presentation,jan'24_presentation,feb'24_presentation,feb_dpd,mar_dpd,apr_dpd,may_dpd,jun_dpd,jul_dpd,aug_dpd,sep_dpd,oct_dpd,nov_dpd,dec_dpd,jan'23_dpd,feb'23_dpd,mar'23_dpd,apr'23_dpd,may'23_dpd,jun'23_dpd,july'23_dpd,aug'23_dpd,sep'23_dpd,oct'23_dpd,nov'23_dpd,dec'23_dpd,jan'24_dpd,mar_dpd_rank,payu'july'23_dpd,payu'aug'23_dpd,payu'sep'23_dpd,payu'oct'23_dpd,payu'nov'23_dpd,payu'dec'23_dpd,payu'jan'24_dpd,sep_dpd_rank,jan_-_pos,feb_-_pos,mar_-_pos,apr_-_pos,may_-_pos,jun_-_pos,jul_-_pos,aug_-_pos,sep_-_pos,oct_-_pos,nov_-_pos,dec_-_pos,jan'23_-_pos,feb'23_-_pos,mar'23_-_pos,apr'23_-_pos,may'23_-_pos,jun'23_-_pos,july'23_-_pos,aug'23_-_pos,sep'23_-_pos,oct'23_-_pos,nov'23_-_pos,dec'23_-_pos,jan'24_-_pos,int_*_loan_amount_(all_incl),cof_*_loan_amount_(all_incl),tenor_*_loan_amount_(all_incl),pf_*_loan_amount_(all_incl),spread_*_loan_amount_(all_incl),foir,ok/problem_areas,loan_count_per_customer,repeat/unique,previous_loan_date_of_repeat,previous_loan_amount_of_repeat,dpd_90+_months,dpd_60+_months,90+_amount,feb_-_repaid,`,apr_-_repaid,may_-_repaid,jun_-_repaid,jul_-_repaid,aug_-_repaid,sep_-_repaid,oct_-_repaid,nov_-_repaid,dec_-_repaid,jan'23_-_repaid,feb'23_-_repaid,mar'23_-_repaid,apr'23_-_repaid,may'23_-_repaid,jun'23_-_repaid,july'23_-_repaid,aug'23_-_repaid,sep'23_-_repaid,unnamed:_176,unnamed:_177,unnamed:_178,unnamed:_179,unnamed:_180,pf,category,principal_amount,interest_rate,attributed_partner,propensity,offer_type,max_amount,income_source,purpose_of_loan,residence_ownership_type,marital_status,cpv_type,lender_name,disbursement_date,acq_cibil_gender,acq_cibil_pincode,acq_cibil_calculated_age,acq_cibil_state,createdate,acq_current_salary,acq_available_income,acq_foir_dlq,acq_foir_cibil,acq_foir_trd,acq_foir_mean,acq_new_foir,acq_naps_score,rnk,disb_dt,disb_yymm,months_since_disb
30658,848f3a4d-0ae6-4dcf-b40e-23e561bdbba1,5a5b1325-c1e9-4d89-a77f-bf18495bdbe4,liquiloans,Organic,6704542,2023-11-30,24,2023,9052848624,Yes,4750,0,4750,855.0,0.0,244395,250000,9,24,4,0.019000,14.98,14.98,3.92,12590.0,80000.0,5,0,0,2,0,777,760+,8,CAT-A,PL,< 24 hrs,K.V.Rangareddy,ANDHRA PRADESH,500074,Tier 1,Male,1,0,30,PL_OLD_PREAPPROVED,REMOTE,Kodimela Rajesh,Active,NaT,NaT,NaN,No,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cleared,Cleared,Bounced,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250000.0,238963.0,230136.667250,4725000.0,3745000.00,6000000,4750.0,980000.00,NaN,NaN,1,Unique,00:00:00,0.0,0,NaN,NaN,NaN,38914.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,0.019000,CAT-A,250000.0,18.9,niro,High,PRE_APPROVED,250000.0,Salaried,Purchase,rented,MARRIED,RE

#### Derived variables

In [21]:
master_data['state'] = master_data['state'].replace(0,None)

In [22]:
master_data['category'].value_counts()

category
CAT-B    13525
CAT-C     8912
CAT-D     6118
CAT-A     2108
Name: count, dtype: int64

In [23]:
master_data.head()

,niro_user_id,niro_opportunity_id,disbursal_partner_name,platform_partner_name,lender_loan_id,disbursement_month,month_number,disbursement_year,phone_numbers,presentation_lag,gross_pf_charged,pf_refund,final_proc_fees,gst,pre-emi,disbursed_amount,loan_amount_(all_inclusive),loan_amount_rank,tenor_in_months,tenor_rank,pf_%,cost_of_funds,cost_of_funds_(new),spread,monthly_emi,monthly_income_(net_take_home),income_rank,self_employed,nature_of_business,#_years_in_employment,owned,cibil_score,cibil_band,cibil_score_rank,risk_band,pl_&_el-pl,disbursement_tat,city,state,pin_code,tier,gender,male,female,age,customer_router,cpv_initiated,lead_name,loan_status,foreclosure_date,foreclosure_details,npa_details,npa_setteled_status,write-off_month,feb_presentation,mar_presentation,apr_presentation,may_presentation,jun_presentation,jul_presentation,aug_presentation,sep_presentation,oct_presentation,nov_presentation,dec_presentation,jan'23_presentation,feb'23_presentation,mar'23_presentation,apr'23_presentation,may'23_presentation,jun'23_presentation,july'23_presentation,aug'23_presentation,sep'23_presentation,oct'23_presentation,nov'23_presentation,dec'23_presentation,jan'24_presentation,feb'24_presentation,feb_dpd,mar_dpd,apr_dpd,may_dpd,jun_dpd,jul_dpd,aug_dpd,sep_dpd,oct_dpd,nov_dpd,dec_dpd,jan'23_dpd,feb'23_dpd,mar'23_dpd,apr'23_dpd,may'23_dpd,jun'23_dpd,july'23_dpd,aug'23_dpd,sep'23_dpd,oct'23_dpd,nov'23_dpd,dec'23_dpd,jan'24_dpd,mar_dpd_rank,payu'july'23_dpd,payu'aug'23_dpd,payu'sep'23_dpd,payu'oct'23_dpd,payu'nov'23_dpd,payu'dec'23_dpd,payu'jan'24_dpd,sep_dpd_rank,jan_-_pos,feb_-_pos,mar_-_pos,apr_-_pos,may_-_pos,jun_-_pos,jul_-_pos,aug_-_pos,sep_-_pos,oct_-_pos,nov_-_pos,dec_-_pos,jan'23_-_pos,feb'23_-_pos,mar'23_-_pos,apr'23_-_pos,may'23_-_pos,jun'23_-_pos,july'23_-_pos,aug'23_-_pos,sep'23_-_pos,oct'23_-_pos,nov'23_-_pos,dec'23_-_pos,jan'24_-_pos,int_*_loan_amount_(all_incl),cof_*_loan_amount_(all_incl),tenor_*_loan_amount_(all_incl),pf_*_loan_amount_(all_incl),spread_*_loan_amount_(all_incl),foir,ok/problem_areas,loan_count_per_customer,repeat/unique,previous_loan_date_of_repeat,previous_loan_amount_of_repeat,dpd_90+_months,dpd_60+_months,90+_amount,feb_-_repaid,`,apr_-_repaid,may_-_repaid,jun_-_repaid,jul_-_repaid,aug_-_repaid,sep_-_repaid,oct_-_repaid,nov_-_repaid,dec_-_repaid,jan'23_-_repaid,feb'23_-_repaid,mar'23_-_repaid,apr'23_-_repaid,may'23_-_repaid,jun'23_-_repaid,july'23_-_repaid,aug'23_-_repaid,sep'23_-_repaid,unnamed:_176,unnamed:_177,unnamed:_178,unnamed:_179,unnamed:_180,pf,category,principal_amount,interest_rate,attributed_partner,propensity,offer_type,max_amount,income_source,purpose_of_loan,residence_ownership_type,marital_status,cpv_type,lender_name,disbursement_date,acq_cibil_gender,acq_cibil_pincode,acq_cibil_calculated_age,acq_cibil_state,createdate,acq_current_salary,acq_available_income,acq_foir_dlq,acq_foir_cibil,acq_foir_trd,acq_foir_mean,acq_new_foir,acq_naps_score,rnk,disb_dt,disb_yymm,months_since_disb
0,e9ed949b-e93b-42a2-ac21-2da7effa5564,8cf95828-6981-4f75-a766-f79769c8bd8a,liquiloans,quikr,518818,2022-01-31,2,2022,9952744141,No,6570,0,6570,1183.0,0.0,180001,187754,7,18,3,0.034993,16.3,16.3,5.2,12295.0,30000.0,3,1,Freelancer,1,0,770,760+,8,CAT-C,PL,< 24 hrs,Chennai,TAMIL NADU,600015,Tier 1,Male,1,0,44,PL_OLD_PREAPPROVED,Yes,SWAMI ELUMALAI GOVINDA MANI,NPA,NaT,NaT,NaN,Yes,2023-03-31 00:00:00,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Bounced,Bounced (DPD),Bounced (DPD),Bounced (DPD),T-BOUNCE,NPA,NPA,NPA,NPA,NPA,NPA,NPA,NPA,NPA,NPA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-29+,30+,60+,90+,NPA,NPA,NPA,NPA,NPA,NPA,NPA,NPA,NPA,NPA,NaN,NPA,NPA,NPA,NPA,NPA,NPA,NPA,NaN,187754.0,178822.925833,169731.836588,160477.865327,151058.093747,141469.551260,131709.214054,121771.348458,111660.788379,101366.788379,90887.788379,90887.788379,90887.788379,90887.788379,90887.788379,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,90887.788379,90887.788379,4036711.0,3

In [24]:
state_region_mapping = {
    'TAMIL NADU': 'South',
    'ANDHRA PRADESH': 'South',
    'KERALA': 'South',
    'KARNATAKA': 'South',
    'MAHARASHTRA': 'West',
    'MADHYA PRADESH': 'Central',
    'DELHI': 'North',
    'UTTAR PRADESH': 'North',
    'PUNJAB': 'North',
    'WEST BENGAL': 'East',
    'GUJARAT': 'West',
    'HARYANA': 'North',
    'RAJASTHAN': 'North',
    'ODISHA': 'East',
    'UTTARAKHAND': 'North',
    'JHARKHAND': 'East',
    'CHANDIGARH': 'North',
    'CHATTISGARH': 'Central',
    'PONDICHERRY': 'South',
    'GOA': 'West',
    'BIHAR': 'East',
    'DADRA & NAGAR HAVELI': 'West',
    'HIMACHAL PRADESH': 'North',
    'DAMAN & DIU': 'West'
}


In [25]:
master_data['region'] = master_data['state'].map(state_region_mapping)

In [26]:
master_data = master_data.apply(pd.to_numeric, errors='ignore')

In [27]:
master_data['monthly_income_(net_take_home)'] = master_data['monthly_income_(net_take_home)'].astype(float)

In [28]:
master_data['offered_la_by_disb_la'] = master_data['max_amount'] / master_data['principal_amount']
master_data['emi_by_income'] = master_data['monthly_emi'] / master_data['monthly_income_(net_take_home)']
master_data['mob_by_tenure'] = master_data['months_since_disb'] / master_data['tenor_in_months']

#### On Us Performance

In [29]:
master_data["delq_bkt"] = np.where(master_data[observation_month + "_dpd"].isin(['-', 'Advance']) | ((pd.to_numeric(master_data[observation_month + "_dpd"], errors='coerce') >= -45) & (pd.to_numeric(master_data[observation_month + "_dpd"], errors='coerce') <= 29)), 'A) Bkt 0',
                                                np.where(master_data[observation_month + "_dpd"].isin(['1-29+']), 'B) Bkt 1',
                                                np.where(master_data[observation_month + "_dpd"].isin(['30+']), 'C) Bkt 2',
                                                np.where(master_data[observation_month + "_dpd"].isin(['60+']), 'D) Bkt 3',
                                                np.where(master_data[observation_month + "_dpd"].isin(['90+']), 'E) Bkt 3+',
                                                np.where(master_data[observation_month + "_dpd"].isin(['NPA (Unsettled)', 'NPA']), 'E) Bkt 3+',
                                                np.where(master_data[observation_month + "_dpd"].isin(['Settled', 'Closed', 'Settled (partial paid)', 'Foreclosed']), 'F) Closed',master_data[observation_month + "_dpd"])))))))

In [30]:
master_data = master_data[~master_data['delq_bkt'].isin(['F) Closed'])].reset_index(drop=True)

##### Delq in last xM

In [31]:
df = master_data.iloc[:,master_data.columns.get_loc("feb_-_pos"):master_data.columns.get_loc(observation_month + "_-_pos")+1]

In [32]:
def backfill_zeros(row):
    start = None
    last_nonzero_value = None
    for i, value in enumerate(row):
        if value != 0:
            last_nonzero_value = value
            start = i
        elif last_nonzero_value is not None:
            break
        
    if start is not None:
        row[start+1:-1] = row[-2]
    return row

df = df.apply(backfill_zeros, axis=1)

In [33]:
master_data.iloc[:,master_data.columns.get_loc("feb_-_pos"):master_data.columns.get_loc(observation_month + "_-_pos")+1] = df

In [34]:
for x in [3,6,12]:
    master_data['bounced_l' + str(x) + 'm'] = master_data.iloc[:,master_data.columns.get_loc(observation_month + "_presentation")-x:master_data.columns.get_loc(observation_month + "_presentation")].isin(['Bounced', 'Bounced (DPD)', 'NPA (unsettled)', 'NPA']).any(axis=1)
    master_data['t_bounced_l' + str(x) + 'm'] = master_data.iloc[:,master_data.columns.get_loc(observation_month + "_presentation")-x:master_data.columns.get_loc(observation_month + "_presentation")].isin(['T-BOUNCE']).any(axis=1)
    master_data['30p_l' + str(x) + 'm'] = master_data.iloc[:,master_data.columns.get_loc(observation_month + "_dpd")-x:master_data.columns.get_loc(observation_month + "_dpd")].isin(['1-29+', '30+', '60+', '90+', 'NPA (unsettled)', 'NPA']).any(axis=1)
    master_data['60p_l' + str(x) + 'm'] = master_data.iloc[:,master_data.columns.get_loc(observation_month + "_dpd")-x:master_data.columns.get_loc(observation_month + "_dpd")].isin(['60+', '90+', 'NPA (unsettled)', 'NPA']).any(axis=1)
    master_data['90p_l' + str(x) + 'm'] = master_data.iloc[:,master_data.columns.get_loc(observation_month + "_dpd")-x:master_data.columns.get_loc(observation_month + "_dpd")].isin(['90+', 'NPA (unsettled)', 'NPA']).any(axis=1)
    master_data['advance_l' + str(x) + 'm'] = master_data.iloc[:,master_data.columns.get_loc(observation_month + "_presentation")-x:master_data.columns.get_loc(observation_month + "_presentation")].isin(['Advance']).any(axis=1)
    
    master_data['max_delq_l' + str(x) + 'm'] = np.where(master_data.iloc[:,master_data.columns.get_loc(observation_month + "_dpd")-x:master_data.columns.get_loc(observation_month + "_dpd")].isin(['NPA (unsettled)', 'NPA']).any(axis=1), 'NPA',
                                                    np.where(master_data.iloc[:,master_data.columns.get_loc(observation_month + "_dpd")-x:master_data.columns.get_loc(observation_month + "_dpd")].isin(['90+']).any(axis=1), '90+',
                                                             np.where(master_data.iloc[:,master_data.columns.get_loc(observation_month + "_dpd")-x:master_data.columns.get_loc(observation_month + "_dpd")].isin(['60+']).any(axis=1), '60+',
                                                                      np.where(master_data.iloc[:,master_data.columns.get_loc(observation_month + "_dpd")-x:master_data.columns.get_loc(observation_month + "_dpd")].isin(['1-29+', '30+']).any(axis=1), '30+', 
                                                                            np.where(master_data.iloc[:,master_data.columns.get_loc(observation_month + "_dpd")-x:master_data.columns.get_loc(observation_month + "_dpd")].isin(['-', 'Advance']).any(axis=1), '0',
                                                                                np.where(master_data.iloc[:,master_data.columns.get_loc(observation_month + "_dpd")-x:master_data.columns.get_loc(observation_month + "_dpd")].apply(pd.to_numeric, errors='coerce').max(axis=1).isin(range(20,30)), '20-28',
                                                                                    np.where(master_data.iloc[:,master_data.columns.get_loc(observation_month + "_dpd")-x:master_data.columns.get_loc(observation_month + "_dpd")].apply(pd.to_numeric, errors='coerce').max(axis=1).isin(range(10,20)), '10-19',
                                                                                        np.where(master_data.iloc[:,master_data.columns.get_loc(observation_month + "_dpd")-x:master_data.columns.get_loc(observation_month + "_dpd")].apply(pd.to_numeric, errors='coerce').max(axis=1).isin(range(1,10)), '1-9',
                                                                                            np.where(master_data.iloc[:,master_data.columns.get_loc(observation_month + "_dpd")-x:master_data.columns.get_loc(observation_month + "_dpd")].apply(pd.to_numeric, errors='coerce').max(axis=1) < 0, '0', 
                                                                                            master_data.iloc[:,master_data.columns.get_loc(observation_month + "_dpd")-x:master_data.columns.get_loc(observation_month + "_dpd")].apply(pd.to_numeric, errors='coerce').max(axis=1))))))))))
    
    master_data['times_bounced_l' + str(x) + 'm'] = master_data.iloc[:,master_data.columns.get_loc(observation_month + "_presentation")-x:master_data.columns.get_loc(observation_month + "_presentation")].isin(['Bounced', 'Bounced (DPD)', 'NPA (unsettled)', 'NPA']).sum(axis=1)
    master_data['times_t_bounced_l' + str(x) + 'm'] = master_data.iloc[:,master_data.columns.get_loc(observation_month + "_presentation")-x:master_data.columns.get_loc(observation_month + "_presentation")].isin(['T-BOUNCE']).sum(axis=1)
    master_data['times_30p_l' + str(x) + 'm'] = master_data.iloc[:,master_data.columns.get_loc(observation_month + "_dpd")-x:master_data.columns.get_loc(observation_month + "_dpd")].isin(['1-29+', '30+', '60+', '90+', 'NPA (unsettled)', 'NPA']).sum(axis=1)
    master_data['times_60p_l' + str(x) + 'm'] = master_data.iloc[:,master_data.columns.get_loc(observation_month + "_dpd")-x:master_data.columns.get_loc(observation_month + "_dpd")].isin(['60+', '90+', 'NPA (unsettled)', 'NPA']).sum(axis=1)
    master_data['times_90p_l' + str(x) + 'm'] = master_data.iloc[:,master_data.columns.get_loc(observation_month + "_dpd")-x:master_data.columns.get_loc(observation_month + "_dpd")].isin(['90+', 'NPA (unsettled)', 'NPA']).sum(axis=1)
    master_data['times_advance_l' + str(x) + 'm'] = master_data.iloc[:,master_data.columns.get_loc(observation_month + "_presentation")-x:master_data.columns.get_loc(observation_month + "_presentation")].isin(['Advance']).sum(axis=1)
    
    master_data['pos_paid_l' + str(x) + 'm'] = master_data.iloc[:,master_data.columns.get_loc(observation_month + "_-_pos")-x] - master_data.iloc[:,master_data.columns.get_loc(observation_month + "_-_pos")]
    master_data['pos_paid_l' + str(x) + 'm_by_la'] = master_data['pos_paid_l' + str(x) + 'm'] / master_data['loan_amount_(all_inclusive)']

master_data['bounced_ever'] = master_data.iloc[:,master_data.columns.get_loc("feb_presentation"):master_data.columns.get_loc(observation_month + "_presentation")].isin(['Bounced', 'Bounced (DPD)', 'NPA (unsettled)', 'NPA']).any(axis=1)
master_data['t_bounced_ever'] = master_data.iloc[:,master_data.columns.get_loc("feb_presentation"):master_data.columns.get_loc(observation_month + "_presentation")].isin(['T-BOUNCE']).any(axis=1)
master_data['30p_ever'] = master_data.iloc[:,master_data.columns.get_loc("feb_presentation"):master_data.columns.get_loc(observation_month + "_dpd")].isin(['1-29+', '30+', '60+', '90+', 'NPA (unsettled)', 'NPA']).any(axis=1)
master_data['60p_ever'] = master_data.iloc[:,master_data.columns.get_loc("feb_presentation"):master_data.columns.get_loc(observation_month + "_dpd")].isin(['60+', '90+', 'NPA (unsettled)', 'NPA']).any(axis=1)
master_data['90p_ever'] = master_data.iloc[:,master_data.columns.get_loc("feb_presentation"):master_data.columns.get_loc(observation_month + "_dpd")].isin(['90+', 'NPA (unsettled)', 'NPA']).any(axis=1)
master_data['advance_ever'] = master_data.iloc[:,master_data.columns.get_loc("feb_presentation"):master_data.columns.get_loc(observation_month + "_presentation")].isin(['Advance']).any(axis=1)

master_data['max_delq_ever'] = np.where(master_data.iloc[:,master_data.columns.get_loc("feb_presentation"):master_data.columns.get_loc(observation_month + "_dpd")].isin(['NPA (unsettled)', 'NPA']).any(axis=1), 'NPA',
                                                    np.where(master_data.iloc[:,master_data.columns.get_loc("feb_presentation"):master_data.columns.get_loc(observation_month + "_dpd")].isin(['90+']).any(axis=1), '90+',
                                                             np.where(master_data.iloc[:,master_data.columns.get_loc("feb_presentation"):master_data.columns.get_loc(observation_month + "_dpd")].isin(['60+']).any(axis=1), '60+',
                                                                      np.where(master_data.iloc[:,master_data.columns.get_loc("feb_presentation"):master_data.columns.get_loc(observation_month + "_dpd")].isin(['1-29+', '30+']).any(axis=1), '30+', 
                                                                            np.where(master_data.iloc[:,master_data.columns.get_loc("feb_presentation"):master_data.columns.get_loc(observation_month + "_dpd")].isin(['-', 'Advance']).any(axis=1), '0',
                                                                                np.where(master_data.iloc[:,master_data.columns.get_loc("feb_presentation"):master_data.columns.get_loc(observation_month + "_dpd")].apply(pd.to_numeric, errors='coerce').max(axis=1).isin(range(20,30)), '20-28',
                                                                                    np.where(master_data.iloc[:,master_data.columns.get_loc("feb_presentation"):master_data.columns.get_loc(observation_month + "_dpd")].apply(pd.to_numeric, errors='coerce').max(axis=1).isin(range(10,20)), '10-19',
                                                                                        np.where(master_data.iloc[:,master_data.columns.get_loc("feb_presentation"):master_data.columns.get_loc(observation_month + "_dpd")].apply(pd.to_numeric, errors='coerce').max(axis=1).isin(range(1,10)), '1-9',
                                                                                            np.where(master_data.iloc[:,master_data.columns.get_loc("feb_presentation"):master_data.columns.get_loc(observation_month + "_dpd")].apply(pd.to_numeric, errors='coerce').max(axis=1) < 0, '0', 
                                                                                            master_data.iloc[:,master_data.columns.get_loc("feb_presentation"):master_data.columns.get_loc(observation_month + "_dpd")].apply(pd.to_numeric, errors='coerce').max(axis=1))))))))))

master_data['times_bounced_ever'] = master_data.iloc[:,master_data.columns.get_loc("feb_presentation"):master_data.columns.get_loc(observation_month + "_presentation")].isin(['Bounced', 'Bounced (DPD)', 'NPA (unsettled)', 'NPA']).sum(axis=1)
master_data['times_t_bounced_ever'] = master_data.iloc[:,master_data.columns.get_loc("feb_presentation"):master_data.columns.get_loc(observation_month + "_presentation")].isin(['T-BOUNCE']).sum(axis=1)
master_data['times_30p_ever'] = master_data.iloc[:,master_data.columns.get_loc("feb_presentation"):master_data.columns.get_loc(observation_month + "_dpd")].isin(['1-29+', '30+', '60+', '90+', 'NPA (unsettled)', 'NPA']).sum(axis=1)
master_data['times_60p_ever'] = master_data.iloc[:,master_data.columns.get_loc("feb_presentation"):master_data.columns.get_loc(observation_month + "_dpd")].isin(['60+', '90+', 'NPA (unsettled)', 'NPA']).sum(axis=1)
master_data['times_90p_ever'] = master_data.iloc[:,master_data.columns.get_loc("feb_presentation"):master_data.columns.get_loc(observation_month + "_dpd")].isin(['90+', 'NPA (unsettled)', 'NPA']).sum(axis=1)
master_data['times_advance_ever'] = master_data.iloc[:,master_data.columns.get_loc("feb_presentation"):master_data.columns.get_loc(observation_month + "_presentation")].isin(['Advance']).sum(axis=1)


master_data['times_bounced_ever_by_tenor'] = (master_data['times_bounced_ever'] / master_data['tenor_in_months'])
master_data['times_t_bounced_ever_by_tenor'] = (master_data['times_t_bounced_ever'] / master_data['tenor_in_months'])
master_data['times_30p_ever_by_tenor'] = (master_data['times_30p_ever'] / master_data['tenor_in_months'])
master_data['times_60p_ever_by_tenor'] = (master_data['times_60p_ever'] / master_data['tenor_in_months'])
master_data['times_90p_ever_by_tenor'] = (master_data['times_90p_ever'] / master_data['tenor_in_months'])
master_data['times_advance_ever_by_tenor'] = (master_data['times_advance_ever'] / master_data['tenor_in_months'])

master_data['pos_paid_by_la'] = (master_data['principal_amount'] - master_data.iloc[:,master_data.columns.get_loc(observation_month + "_-_pos")]) / master_data['principal_amount']
master_data['pos_paid_l3m_by_la'] = master_data['pos_paid_l3m'] / master_data['principal_amount']
master_data['pos_paid_l6m_by_la'] = master_data['pos_paid_l6m'] / master_data['principal_amount']

master_data['bounced_current_month'] = master_data.iloc[:,master_data.columns.get_loc(observation_month + "_presentation")].isin(['Bounced', 'Bounced (DPD)', 'NPA (unsettled)', 'NPA'])
master_data['bounced_current_month_non_dpd'] = master_data.iloc[:,master_data.columns.get_loc(observation_month + "_presentation")].isin(['Bounced', 'Bounced (DPD)', 'NPA (unsettled)', 'NPA']) & master_data.iloc[:,master_data.columns.get_loc(observation_month + "_dpd")].isin(['-', '0'])
master_data['advance_current_month'] = master_data.iloc[:,master_data.columns.get_loc(observation_month + "_presentation")].isin(['Advance'])

master_data['times_bounced_l3m_by_times_bounced_l3_6m'] = master_data['times_bounced_l3m']/(master_data['times_bounced_l6m'] - master_data['times_bounced_l3m'])
master_data['times_bounced_l6m_by_times_bounced_l6_12m'] = master_data['times_bounced_l6m']/(master_data['times_bounced_l12m'] - master_data['times_bounced_l6m'])

master_data['pos_paid_l3m_by_pos_paid_l3_6m'] = master_data['pos_paid_l3m']/(master_data['pos_paid_l6m'] - master_data['pos_paid_l3m'])
master_data['pos_paid_l6m_by_pos_paid_l6_12m'] = master_data['pos_paid_l6m']/(master_data['pos_paid_l12m'] - master_data['pos_paid_l6m'])

In [35]:
master_data.iloc[:,master_data.columns.get_loc(observation_month + "_presentation")].value_counts()

jan'24_presentation
Cleared            19876
Bounced             5325
Advance             1644
Bounced (DPD)       1626
NPA (unsettled)     1364
NPA                  611
T-BOUNCE             217
Name: count, dtype: int64

In [36]:
master_data.loc[master_data['max_delq_ever'] == 'nan'].head()

,niro_user_id,niro_opportunity_id,disbursal_partner_name,platform_partner_name,lender_loan_id,disbursement_month,month_number,disbursement_year,phone_numbers,presentation_lag,gross_pf_charged,pf_refund,final_proc_fees,gst,pre-emi,disbursed_amount,loan_amount_(all_inclusive),loan_amount_rank,tenor_in_months,tenor_rank,pf_%,cost_of_funds,cost_of_funds_(new),spread,monthly_emi,monthly_income_(net_take_home),income_rank,self_employed,nature_of_business,#_years_in_employment,owned,cibil_score,cibil_band,cibil_score_rank,risk_band,pl_&_el-pl,disbursement_tat,city,state,pin_code,tier,gender,male,female,age,customer_router,cpv_initiated,lead_name,loan_status,foreclosure_date,foreclosure_details,npa_details,npa_setteled_status,write-off_month,feb_presentation,mar_presentation,apr_presentation,may_presentation,jun_presentation,jul_presentation,aug_presentation,sep_presentation,oct_presentation,nov_presentation,dec_presentation,jan'23_presentation,feb'23_presentation,mar'23_presentation,apr'23_presentation,may'23_presentation,jun'23_presentation,july'23_presentation,aug'23_presentation,sep'23_presentation,oct'23_presentation,nov'23_presentation,dec'23_presentation,jan'24_presentation,feb'24_presentation,feb_dpd,mar_dpd,apr_dpd,may_dpd,jun_dpd,jul_dpd,aug_dpd,sep_dpd,oct_dpd,nov_dpd,dec_dpd,jan'23_dpd,feb'23_dpd,mar'23_dpd,apr'23_dpd,may'23_dpd,jun'23_dpd,july'23_dpd,aug'23_dpd,sep'23_dpd,oct'23_dpd,nov'23_dpd,dec'23_dpd,jan'24_dpd,mar_dpd_rank,payu'july'23_dpd,payu'aug'23_dpd,payu'sep'23_dpd,payu'oct'23_dpd,payu'nov'23_dpd,payu'dec'23_dpd,payu'jan'24_dpd,sep_dpd_rank,jan_-_pos,feb_-_pos,mar_-_pos,apr_-_pos,may_-_pos,jun_-_pos,jul_-_pos,aug_-_pos,sep_-_pos,oct_-_pos,nov_-_pos,dec_-_pos,jan'23_-_pos,feb'23_-_pos,mar'23_-_pos,apr'23_-_pos,may'23_-_pos,jun'23_-_pos,july'23_-_pos,aug'23_-_pos,sep'23_-_pos,oct'23_-_pos,nov'23_-_pos,dec'23_-_pos,jan'24_-_pos,int_*_loan_amount_(all_incl),cof_*_loan_amount_(all_incl),tenor_*_loan_amount_(all_incl),pf_*_loan_amount_(all_incl),spread_*_loan_amount_(all_incl),foir,ok/problem_areas,loan_count_per_customer,repeat/unique,previous_loan_date_of_repeat,previous_loan_amount_of_repeat,dpd_90+_months,dpd_60+_months,90+_amount,feb_-_repaid,`,apr_-_repaid,may_-_repaid,jun_-_repaid,jul_-_repaid,aug_-_repaid,sep_-_repaid,oct_-_repaid,nov_-_repaid,dec_-_repaid,jan'23_-_repaid,feb'23_-_repaid,mar'23_-_repaid,apr'23_-_repaid,may'23_-_repaid,jun'23_-_repaid,july'23_-_repaid,aug'23_-_repaid,sep'23_-_repaid,unnamed:_176,unnamed:_177,unnamed:_178,unnamed:_179,unnamed:_180,pf,category,principal_amount,interest_rate,attributed_partner,propensity,offer_type,max_amount,income_source,purpose_of_loan,residence_ownership_type,marital_status,cpv_type,lender_name,disbursement_date,acq_cibil_gender,acq_cibil_pincode,acq_cibil_calculated_age,acq_cibil_state,createdate,acq_current_salary,acq_available_income,acq_foir_dlq,acq_foir_cibil,acq_foir_trd,acq_foir_mean,acq_new_foir,acq_naps_score,rnk,disb_dt,disb_yymm,months_since_disb,region,offered_la_by_disb_la,emi_by_income,mob_by_tenure,delq_bkt,bounced_l3m,t_bounced_l3m,30p_l3m,60p_l3m,90p_l3m,advance_l3m,max_delq_l3m,times_bounced_l3m,times_t_bounced_l3m,times_30p_l3m,times_60p_l3m,times_90p_l3m,times_advance_l3m,pos_paid_l3m,pos_paid_l3m_by_la,bounced_l6m,t_bounced_l6m,30p_l6m,60p_l6m,90p_l6m,advance_l6m,max_delq_l6m,times_bounced_l6m,times_t_bounced_l6m,times_30p_l6m,times_60p_l6m,times_90p_l6m,times_advance_l6m,pos_paid_l6m,pos_paid_l6m_by_la,bounced_l12m,t_bounced_l12m,30p_l12m,60p_l12m,90p_l12m,advance_l12m,max_delq_l12m,times_bounced_l12m,times_t_bounced_l12m,times_30p_l12m,times_60p_l12m,times_90p_l12m,times_advance_l12m,pos_paid_l12m,pos_paid_l12m_by_la,bounced_ever,t_bounced_ever,30p_ever,60p_ever,90p_ever,advance_ever,max_delq_ever,times_bounced_ever,times_t_bounced_ever,times_30p_ever,times_60p_ever,times_90p_ever,times_advance_ever,times_bounced_ever_by_tenor,times_t_bounced_ever_by_tenor,times_30p_ever_by_tenor,times_60p_ever_by_tenor,times_90p_ever_by_tenor,times_adv

In [37]:
master_data.iloc[:,master_data.columns.get_loc(observation_month + "_dpd")].isin(['-', '0'])

0        False
1        False
2         True
3        False
4        False
         ...  
30658     True
30659     True
30660     True
30661     True
30662    False
Name: jan'24_dpd, Length: 30663, dtype: bool

#### Derived variables

In [38]:
master_data.columns = master_data.columns.str.replace('[^a-zA-Z0-9]', '_')
master_data.columns = master_data.columns.str.replace(' ', '_') 
master_data.columns = master_data.columns.str.lower()

In [39]:
new_cols = list(set(master_data.columns) - set(master_calling.columns))
new_cols

['90p_ever',
 'category',
 'times_90p_l6m',
 '30p_l6m',
 'pos_paid_l12m_by_la',
 'pos_paid_l6m_by_pos_paid_l6_12m',
 'max_amount',
 'max_delq_l3m',
 'times_advance_l12m',
 'propensity',
 'acq_cibil_state',
 '90p_l3m',
 'cpv_type',
 'offered_la_by_disb_la',
 'acq_current_salary',
 'times_bounced_l3m',
 'times_90p_ever_by_tenor',
 'times_advance_ever_by_tenor',
 'lender_name',
 'mob_by_tenure',
 'acq_new_foir',
 'disbursement_date',
 '60p_l6m',
 'delq_bkt',
 'times_t_bounced_ever',
 'times_bounced_l6m_by_times_bounced_l6_12m',
 '30p_l12m',
 'bounced_l12m',
 'times_t_bounced_l6m',
 'pos_paid_l12m',
 't_bounced_l3m',
 'region',
 '30p_l3m',
 '60p_l12m',
 'times_bounced_ever',
 'times_bounced_l3m_by_times_bounced_l3_6m',
 'purpose_of_loan',
 'times_30p_l6m',
 'pos_paid_l3m',
 'attributed_partner',
 'times_t_bounced_l3m',
 'interest_rate',
 'times_bounced_l12m',
 't_bounced_l12m',
 'pos_paid_by_la',
 'acq_foir_mean',
 'times_60p_l12m',
 'times_60p_l6m',
 'months_since_disb',
 'times_t_bounced

In [40]:
loan_book_cols = ['niro_opportunity_id', 'cibil_score', 'foir', 'gender', 'ok/problem_areas', 'repeat/unique', 'risk_band', 'tier', 'emi', observation_month + "_-_pos"]

#### Event var preparation

In [41]:
master_data['next_month_bounce'] = master_data.iloc[:,master_data.columns.get_loc(observation_month + "_presentation")+1].isin(['Bounced', 'Bounced (DPD)', 'NPA (unsettled)', 'NPA'])

In [42]:
master_data.iloc[:,master_data.columns.get_loc(observation_month + "_dpd")]

0                    NPA
1        NPA (Unsettled)
2                      -
3                    90+
4                    NPA
              ...       
30658                  -
30659                  -
30660                  -
30661                  -
30662                  1
Name: jan'24_dpd, Length: 30663, dtype: object

In [43]:
master_data['next_month_bounce'].value_counts()

next_month_bounce
False    21466
True      9197
Name: count, dtype: int64

In [44]:
master_data['next_month_bounce'].value_counts(normalize=True)

next_month_bounce
False    0.700062
True     0.299938
Name: proportion, dtype: float64

In [45]:
master_data[master_data["delq_bkt"] == 'A) Bkt 0']['next_month_bounce'].value_counts()

next_month_bounce
False    21279
True      5014
Name: count, dtype: int64

In [46]:
master_data[master_data["delq_bkt"] == 'A) Bkt 0']['next_month_bounce'].value_counts(normalize=True)

next_month_bounce
False    0.809303
True     0.190697
Name: proportion, dtype: float64

In [47]:
master_data[master_data["delq_bkt"] != 'A) Bkt 0']['next_month_bounce'].value_counts()

next_month_bounce
True     4183
False     187
Name: count, dtype: int64

In [48]:
master_data[master_data["delq_bkt"] != 'A) Bkt 0']['next_month_bounce'].value_counts(normalize=True)

next_month_bounce
True     0.957208
False    0.042792
Name: proportion, dtype: float64

In [49]:
# master_data = master_data[loan_book_cols + new_cols]

#### Variable distribution check

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming master_data is your DataFrame
# Replace 'YourCategoricalColumn' and 'YourNumericalColumn' with the actual column names

# Display value counts for categorical columns
categorical_columns = master_data.select_dtypes(include=['object']).columns
for col in categorical_columns:
    print(f"Value counts for {col}:")
    print(master_data[col].value_counts())
    print("\n")

# Display separate histograms for numerical columns
numerical_columns = master_data.select_dtypes(include=['int64', 'float64']).columns
for col in numerical_columns:
    print(col)
    plt.figure(figsize=(8, 6))
    master_data[col].hist(bins=20)
    plt.title(f'Histogram for {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.show()


In [ ]:
# master_data.to_pickle('master_data.pkl')

In [ ]:
master_data.to_